In [ ]:
import pandas as pd
import numpy as np
import my_modules

c:\Users\ken05\Documents\others\HORSE_RACING\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
names = [
    "race_id",
    "year",
    "month",
    "day",
    "times",
    "place",
    "daily",
    "race_num",
    "horse",
    "jockey_id",
    "horse_N",
    "waku_num",
    "horse_num",
    "class_code",
    "track_code",
    "corner_num",
    "dist",
    "state",
    "weather",
    "age_code",
    "sex",
    "age",
    "basis_weight",
    "blinker",
    "weight",
    "inc_dec",
    "weight_code",
    "win_odds",
    "rank",
    "time_diff",
    "time",
    "corner1_rank",
    "corner2_rank",
    "corner3_rank",
    "corner4_rank",
    "last_3F_time",
    "last_3F_rank",
    "Ave_3F",
    "PCI",
    "last_3F_time_diff",
    "leg",
    "pop",
    "prize",
    "error_code",
    "father",
    "mother",
    "id"
]


In [3]:
df = pd.read_csv("record_data_2023.csv", encoding="shift-jis", header=None, names=names)
df.head()

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,...,Ave_3F,PCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,id
0,202307220101010101,23,7,22,1,札幌,1,1,ウィスピースノー,1095,...,35.8,53.8,0.8,後方,5.0,55,0,ワールドエース,ハイリマイリ,2021100648
1,202307220101010102,23,7,22,1,札幌,1,1,ロードスタウト,1157,...,35.6,51.4,0.6,中団,7.0,0,0,ロードカナロア,フィラデルフィア,2021100159
2,202307220101010103,23,7,22,1,札幌,1,1,コミックガール,1197,...,35.6,50.8,0.6,中団,4.0,0,0,シルバーステート,コイクレナイ,2021100265
3,202307220101010104,23,7,22,1,札幌,1,1,デビルシズカチャン,5339,...,35.3,51.1,0.3,先行,3.0,83,0,ベストウォーリア,シシリアンブリーズ,2021105553
4,202307220101010105,23,7,22,1,札幌,1,1,サトミノキラリ,1170,...,35.2,52.6,0.2,先行,1.0,550,0,ビッグアーサー,パレード,2021101429


In [4]:
df = my_modules.common_process(df)

c:\Users\ken05\Documents\others\HORSE_RACING\my_modules.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id_for_fold"] = df["race_id"] // 100 # 下二桁を捨てる
c:\Users\ken05\Documents\others\HORSE_RACING\my_modules.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id_for_fold"] = df["id_for_fold"].astype("category")
c:\Users\ken05\Documents\others\HORSE_RACING\my_modules.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [ ]:
import pandas as pd
import numpy as np
from my_modules import GroupTimeSeriesSplit
import optuna
from sklearn.metrics import log_loss
import lightgbm as lgb
from lightgbm import LGBMClassifier

def simple_lightGBM(df):
    splitter = GroupTimeSeriesSplit(n_splits=5)

    # データの前処理
    cat_col = df.select_dtypes(include=["object"]).columns.tolist()
    for col in cat_col:
        df[col] = df[col].astype("category")

    # 訓練データ（パラメータチューニング用）とテストデータの分割
    feature_col = ["waku_num", "horse_num", "sex", "age", "basis_weight", "blinker", "weight", "inc_dec", "mother", "father", "id_for_fold", "horse_N"]
    X, y = df[feature_col], df["target"]  # datetime は学習できないので消す。
    X_train, X_test, y_train, y_test = train_test_group_split(X, y)

    objective = create_objective(X_train, y_train, splitter=splitter)
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.RandomSampler()) # パラメータ探索範囲の確定のため、ランダムサンプラーにする。
    study.optimize(objective, n_trials=100)

    print("Best params : ", study.best_params)

    # 可視化
    optuna.visualization.plot_param_importances(study).show() # feature_importanceみたいなやつ
    optuna.visualization.plot_slice(
        study,
        params=["num_leaves"]
    ).show()


def create_objective(X, y, splitter):
    def objective(trial):
        params = {
            "objective" : "binary",
            "metric" : "binary_logloss",
            "n_estimators": 1000,
            "n_jobs" : -1,
            "verbose" : -1,
            # ここからが調整用パラメータ
            "num_leaves": trial.suggest_int("num_leaves", 2, 100)
        }

        oof_preds = np.zeros(len(X))

        for tr_idx, val_idx in splitter.split(X, y, groups=X["id_for_fold"]):

            X_train, y_train = X.iloc[tr_idx, :], y.iloc[tr_idx]
            X_test, y_test = X.iloc[val_idx, :], y.iloc[val_idx]

            # 要らない列を削除
            X_train, X_test = X_train.drop(["id_for_fold", "horse_N"], axis=1), X_test.drop(["id_for_fold", "horse_N"], axis=1)

            model = LGBMClassifier(**params)
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)] # 30ラウンドで停止

            model.fit(X_train, y_train, 
                      eval_set=[(X_test, y_test)], # 評価データセットを指定
                      eval_metric="binary_logloss",
                      callbacks=callbacks)

                        # 予測値の取得
            oof_preds[val_idx] = model.predict_proba(X_test)[:, 1]
        

        # NaN ではない部分のインデックスを取得
        not_nan_indices = ~np.isnan(oof_preds)
        
        # NaN ではない部分のみで log_loss を計算
        # y も対応するインデックスでフィルタリングする
        avg_logloss = log_loss(y[not_nan_indices], oof_preds[not_nan_indices])
        
        return avg_logloss

    return objective



# グループごとの訓練・テストデータの分割
def train_test_group_split(X, y, test_size=0.2, groups="id_for_fold"):
    
    unique_group = X[groups].unique()
    n_groups = X[groups].nunique()
    n_train = round(n_groups * (1-test_size))
    s_groups = pd.Series(X[groups])

    train_groups = unique_group[:n_train]
    test_groups = unique_group[n_train:]

    train_idx = np.where(s_groups.isin(train_groups))[0]
    test_idx = np.where(s_groups.isin(test_groups))[0]

    X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]

    return X_train, X_test, y_train, y_test

In [7]:
simple_lightGBM(df)

[I 2025-05-26 00:24:59,815] A new study created in memory with name: no-name-a599c467-ef3f-4a84-b282-18057bd4156c
[I 2025-05-26 00:25:00,283] Trial 0 finished with value: 0.6456397694618582 and parameters: {'num_leaves': 55}. Best is trial 0 with value: 0.6456397694618582.
[I 2025-05-26 00:25:00,582] Trial 1 finished with value: 0.645620030530373 and parameters: {'num_leaves': 11}. Best is trial 1 with value: 0.645620030530373.
[I 2025-05-26 00:25:01,076] Trial 2 finished with value: 0.6459963709826954 and parameters: {'num_leaves': 69}. Best is trial 1 with value: 0.645620030530373.
[I 2025-05-26 00:25:01,411] Trial 3 finished with value: 0.645328424581152 and parameters: {'num_leaves': 22}. Best is trial 3 with value: 0.645328424581152.
[I 2025-05-26 00:25:01,932] Trial 4 finished with value: 0.6458682303463572 and parameters: {'num_leaves': 63}. Best is trial 3 with value: 0.645328424581152.
[I 2025-05-26 00:25:02,328] Trial 5 finished with value: 0.645894755932762 and parameters: {

Best params :  {'num_leaves': 21}
